The purpose of this code is to get overall sentiment score of the daily news so as to compare with aspect sentiment score

Its best to run the code in KAGGLE. THE LINK TO THE CODE IS https://www.kaggle.com/code/adi253/sentiment-score

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/newdata/TATA_grouped.csv")  # can change to other csvs 

In [ ]:
df.head()

df.dropna(inplace=True) # dropping null values     

df.reset_index(drop=True, inplace=True)# when u drop the row index would be still there 

In [ ]:
len(df)

In [ ]:
df["news"][375]

In [ ]:
import nltk

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert") # load the tokenizer
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert") # load the model 

Creating two columns for predicted sentiment(either positive negative or neutral) and Sentiment score/probability

In [ ]:
df['Predicted Sentiment'] = None   
df['Sentiment Probability'] = None

The purpose of the below block is to run the sequence model for classification which is trained on finance dataset.

In [ ]:
import torch  # Import PyTorch for inference
import scipy  # Import SciPy for softmax function

# Keyword arguments for tokenizing the input text:
# - padding=True: Pads the input to the maximum length if necessary
# - truncation=True: Truncates the input if it exceeds the maximum length
# - max_length=512: Ensures that no input sequence exceeds 512 tokens
tokenizer_kwargs = {"padding": True, "truncation": True, "max_length": 512}

# Loop over each row in the DataFrame `df`
for i in range(len(df)):
    print(i)  # Print the current row index for tracking progress
    x = df.loc[i, "news"]  # Extract the "news" column value (text) for the current row
    
    # Perform inference without calculating gradients (saves memory and speeds up execution)
    with torch.no_grad():
        # Tokenize the input text using the tokenizer with the specified arguments
        input_sequence = tokenizer(x, return_tensors="pt", **tokenizer_kwargs)
        
        # Pass the tokenized input through the model to get the output
        output = model(**input_sequence)
        
        # Extract the logits (raw output values) from the model
        logits = model(**input_sequence).logits
        
        # Compute softmax probabilities for each label using SciPy's softmax function
        # `model.config.id2label` maps label IDs to label names
        scores = {
            k: v
            for k, v in zip(
                model.config.id2label.values(),  # Convert label IDs to label names
                scipy.special.softmax(logits.numpy().squeeze()),  # Apply softmax to logits and squeeze the tensor
            )
        }
    
    # Get the label with the highest score (predicted sentiment)
    sentiment = max(scores, key=scores.get)
    
    # Get the highest probability associated with the predicted sentiment
    probability = max(scores.values())
    
    # Store the predicted sentiment in the DataFrame under the "Predicted Sentiment" column
    df.loc[i, "Predicted Sentiment"] = sentiment
    
    # Store the probability of the predicted sentiment in the "Sentiment Probability" column
    df.loc[i, "Sentiment Probability"] = probability


In [ ]:
df.head()

In [ ]:
df["Predicted Sentiment"].value_counts()

In [ ]:
df.to_csv("/kaggle/working/tatasentiment.csv")